In [2]:
import nltk
import ssl

In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

##### multiclass problem, need to re-tokenizing long phrase

To do list : check each model + word2vec + chinese tokenization

In [3]:
import jieba
#import jieba.analyse

import gensim
import pickle
import os
import imblearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer, HashingVectorizer
#from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import matplotlib.pyplot as plt 

from sklearn.pipeline import Pipeline
import joblib

from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

os.chdir('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive')
from NLP_function import *
os.chdir('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/In_Database_cn') 

# Import and Split

In [28]:
Df = pd.read_csv('C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/Data_Prepare/Training_data_final.csv', encoding='ANSI')

# remove class without information
Df = Df.fillna('无')
Df = Df[Df.y_cn!='无']
Df = Df[Df.y_cn!='正常']

Df = Df.drop_duplicates(subset='x_cn')

# remove less informative class
#Df = Df[Df.y_cn_count>=10]

In [31]:
Df

,index,x_cn,y_cn,tag,tag_class,check_group,change,y_cn_count,HP
0,4534,I型糖尿病,1型糖尿病,无,无,9,无,4,HP:0100651
1,4535,青少年糖尿病,1型糖尿病,无,无,9,无,4,HP:0100651
2,4536,一型糖尿病,1型糖尿病,无,无,9,无,4,HP:0100651
3,4537,胰岛素 - 依赖性糖尿病,1型糖尿病,无,无,9,无,4,HP:0100651
4,4538,全部卵异常 蜡样透明带 厚而致密 卵周无间隙,Abnormal zona pellucida morphology,无,无,9,无,7,HP:0020156
...,...,...,...,...,...,...,...,...,...
5414,6034,患者 手六趾,足多趾,无,无,9,无,5,HP:0001829
5415,6035,脚部多指,足多趾,无,无,9,无,5,HP:0001829
5416,6036,脚趾骨头复制,足多趾,无,无,9,无,5,HP:0001829
5417,6037,脑梗,卒中,无,无,9,无,2,HP:0001297


In [30]:
# change number
#Df['x_cn'] = Df['x_cn'].apply(lambda x : clean_text(x,lan='cn'))
Df['x_cn'] = Df['x_cn'].apply(lambda x : change_number(x,'cn'))

# define X,Y for model learning
X, Y  = Df['x_cn'], Df['y_cn']
x_cut = X.apply(lambda txt: list(jieba.cut(txt))) #jieba tokenization
x_cut.head()

0                                          [I, 型, 糖尿病]
1                                           [青少年, 糖尿病]
2                                            [一型, 糖尿病]
3                             [胰岛素,  , -,  , 依赖性, 糖尿病]
4    [全部, 卵, 异常,  , 蜡样, 透明, 带,  , 厚, 而, 致密,  , 卵周, ...
Name: x_cn, dtype: object

In [32]:
Df['x_cn']

0                        I型糖尿病
1                       青少年糖尿病
2                        一型糖尿病
3                 胰岛素 - 依赖性糖尿病
4       全部卵异常 蜡样透明带 厚而致密 卵周无间隙
                 ...          
5414                    患者 手六趾
5415                      脚部多指
5416                    脚趾骨头复制
5417                        脑梗
5418                        中风
Name: x_cn, Length: 4287, dtype: object

In [33]:
# Split data into train and test

x_train, x_test, y_train, y_test = train_test_split(
    x_cut, Y, test_size=0.20, random_state = 42)
my_tags = pd.unique(y_test).tolist()

#print('x_train shape:', x_train.shape, 'y_train shape:', y_train.shape, 'x_test shape:', x_test.shape, 'y_test shape:', y_test.shape)
#print('levels of y_test :', len(pd.unique(y_test))) 
#print('levels of y_train :', len(pd.unique(y_train))) 


### Split data into dev (train + cv) and test
#x_train, x_validate, x_test = np.split(x_cut.sample(frac=1), [int(.8*len(x_cut)), int(.9*len(x_cut))])
#y_train, y_validate, y_test = np.split(Y.sample(frac=1), [int(.8*len(Y)), int(.9*len(Y))])
#print('x_train shape:', x_train.shape, 'y_train shape:', y_train.shape, 'x_validate shape:', x_validate.shape, 'y_validate shape:', y_validate.shape,'x_test shape:', x_test.shape, 'y_test shape:', y_test.shape)

In [34]:
from keras.preprocessing import text, sequence
vocab_size  = 2000 
tokenize = text.Tokenizer(num_words = vocab_size, char_level=False)

tokenize.fit_on_texts(x_train) # only fit on train
X_train = tokenize.texts_to_matrix(x_train, mode='tfidf')
#X_validate  = tokenize.texts_to_matrix(x_validate, mode='tfidf')
X_test  = tokenize.texts_to_matrix(x_test, mode='tfidf')

print('X_train shape:', X_train.shape, 'x_train shape:', y_train.shape)
#print('X_validate shape:', X_validate.shape, 'Y_validate shape:', y_validate.shape)
print('X_test shape:', X_test.shape, 'y_test shape:', y_test.shape)

with open('tokenizer_keras_cn_skl.pickle', 'wb') as handle:
    pickle.dump(tokenize, handle, protocol=pickle.HIGHEST_PROTOCOL)

X_train shape: (3429, 2000) x_train shape: (3429,)
X_test shape: (858, 2000) y_test shape: (858,)


In [ ]:
# Increase sample size (in our case do not change evaluation metrics)

from imblearn.over_sampling import SMOTE, ADASYN
smote = SMOTE('minority')

X_sm, y_sm= smote.fit_resample(X_train, y_train)
print('X_sm shape:', X_sm.shape, 'x_sm shape:', y_sm.shape)

### Compare models

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


heldout = [0.95, 0.90, 0.75, 0.50, 0.01]
rounds  = 20

classifiers = [
   #("SGD", SGDClassifier(max_iter=100)), # 0.76
   ("ASGD", SGDClassifier(average=True)), # 0.79
   #("Perceptron", Perceptron()), # 0.79
   ("Passive-Aggressive I", PassiveAggressiveClassifier(loss='hinge', C=1.0, tol=1e-4)), # 0.83
   #("Passive-Aggressive II", PassiveAggressiveClassifier(loss='squared_hinge', C=1.0, tol=1e-4)), # 0.75
   #("SAG", LogisticRegression(solver='sag', tol=1e-1, C=1.e4 / X.shape[0])), # 0.77
    
    
   #('NB', MultinomialNB()),                        # 0.71
   #("Nearest Neighbors", KNeighborsClassifier(3)), # 0.79
    ("Linear SVM", SVC(kernel="linear", C= 1)),  # 0.852
   #('RBF SVM', SVC(gamma=2, C=1)),                 # 0.39
   #('Gaussian Process', GaussianProcessClassifier(1.0 * RBF(1.0))),   
   #('Random Forest', RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0)),   # 0.33
   #("Decision Tree", DecisionTreeClassifier(max_depth=5)), # 0.51
   ("Neural Net", MLPClassifier(alpha=1, max_iter=1000))  # 0.852
   # ("AdaBoost", AdaBoostClassifier()), # 0.36
   # ("QDA", QuadraticDiscriminantAnalysis()),      
]

xx = 1. - np.array(heldout)

for name, clf in classifiers:
    print("training %s" % name)
    rng = np.random.RandomState(42)
    yy = []
    for i in heldout:
        yy_ = []
        for r in range(rounds):
            
            x_train, x_test, y_train, y_test = \
            train_test_split(x_cut, Y, test_size=i, random_state=rng)
            tokenize.fit_on_texts(x_train) # only fit on train
            X_train = tokenize.texts_to_matrix(x_train, mode='tfidf')
            X_test  = tokenize.texts_to_matrix(x_test, mode='tfidf')
            
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            yy_.append(1 - np.mean(y_pred == y_test))
        yy.append(np.mean(yy_)) 
    plt.plot(xx, yy, label=name)
    
plt.legend(loc ="upper right")
plt.xlabel("Proportion train")
plt.ylabel("Test Error Rate")
plt.show()

In [ ]:
### Confusion Matrix

import seaborn as sns

conf_mat = metrics.confusion_matrix(y_test, pd.DataFrame(y_pred))
fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=y_test.drop_duplicates().values, yticklabels=y_test.drop_duplicates().values,
            center=True)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

### Parameter grid for grid search

In [ ]:
#MLPClassifier()
#PassiveAggressiveClassifier()
MLPClassifier() 

In [44]:
# search combination of all hyperparametres (alternatives : RandomizedSearchCV() )

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import make_scorer

#scoring= {'f1_sccore': 'f1_weighted', 'Accuracy': make_scorer(metrics.accuracy_score)}

param_grid = [
  #{'C': [ 0.005, 0.055, 0.08, 0.1, 1, 10], 'kernel': ['linear']},
  {'alpha': [0.1, 0.2, 0.5, 1], 'max_iter': [1000]},  # for MLP
  # {'C': [ 0.005, 0.015, 0.055, 0.1, 0.3, 0.6, 0.8, 1, 10], 'loss': ['hinge']},
 ]

#clf = SVC()
#clf = PassiveAggressiveClassifier()
clf = MLPClassifier() 
search = GridSearchCV(clf, param_grid, cv=5, n_jobs=-1)
search.fit(X_train, y_train)

print("Best estimator found by grid search:")
print(search.best_estimator_)
#print(search.score(X_validate, y_validate))

C:\Users\Jingz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Best estimator found by grid search:
MLPClassifier(activation='relu', alpha=0.2, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)


### Fit models

In [8]:
from sklearn.svm import SVC

# support vector machine
svm = Pipeline([
                ('clf', SVC(kernel="linear", C = 0.055)),
               ])
# svm.fit(X_sm, y_sm)
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
my_tags = pd.unique(y_pred).tolist()
    
# result metrics
print('accuracy %s' % metrics.accuracy_score(y_pred, y_test))   
print('recall %s'   % metrics.recall_score(y_pred, y_test,  average='weighted'))      #, labels=np.unique(y_pred)   # tp / (tp + fn)
print('precision %s' % metrics.precision_score(y_pred, y_test,  average='weighted'))   # tp / (tp + fp)
print('f1_sccore %s' % metrics.f1_score(y_pred, y_test,  average='weighted'))          # 2*(precision * recall) / (precision + recall)


# save metric results into excel
data = pd.DataFrame(metrics.classification_report(y_test, y_pred, output_dict=True)).T
#print(data)
data.to_excel('training_rapport_svm.xlsx', header=True, index=True, encoding = 'ANSI')


# save model
with open('M_svm_classifier_cn.pickle', 'wb') as handle:
    pickle.dump(svm, handle, protocol=pickle.HIGHEST_PROTOCOL)

accuracy 0.7354312354312355
recall 0.7354312354312355
precision 0.8262218060356815
f1_sccore 0.7511894605395296


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: Undef

In [9]:
### cross validation

from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

clf = SVC(kernel="linear", C = 0.055)
#clf = MLPClassifier(alpha=0.02, max_iter=1000)

scores_a = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')
scores_f1 = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_weighted')

print("Accuracy: %0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() * 2))
print("F1: %0.2f (+/- %0.2f)" % (scores_f1.mean(), scores_f1.std() * 2))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Accuracy: 0.73 (+/- 0.02)
F1: 0.70 (+/- 0.03)


### Other models

In [10]:
from sklearn.neural_network import MLPClassifier


# Passive-Aggressive I
mlp = Pipeline([
                ('clf', MLPClassifier(alpha=0.02, max_iter=1000)),
               ])
mlp.fit(X_train, y_train)

y_pred = mlp.predict(X_test)
my_tags = pd.unique(y_pred).tolist()


# result metrics
print('accuracy %s' % metrics.accuracy_score(y_pred, y_test))   
print('recall %s'   % metrics.recall_score(y_pred, y_test,  average='weighted'))      #, labels=np.unique(y_pred)   # tp / (tp + fn)
print('precision %s' % metrics.precision_score(y_pred, y_test,  average='weighted'))   # tp / (tp + fp)
print('f1_sccore %s' % metrics.f1_score(y_pred, y_test,  average='weighted'))          # 2*(precision * recall) / (precision + recall)


# save metric results into excel
print('accuracy %s' %  metrics.accuracy_score(y_pred, y_test))
data = pd.DataFrame( metrics.classification_report(y_test, y_pred, output_dict=True)).T
print(data)
data.to_excel('training_rapport_mlp.xlsx', header=True, index=True, encoding = 'ANSI')


# save model
with open('M_mlp_classifier_cn.pickle', 'wb') as handle:
    pickle.dump(mlp, handle, protocol=pickle.HIGHEST_PROTOCOL)  

accuracy 0.7424242424242424
recall 0.7424242424242424
precision 0.7816016229852364
f1_sccore 0.7457115512389201
accuracy 0.7424242424242424
                                    precision    recall  f1-score     support
Abnormal zona pellucida morphology   1.000000  1.000000  1.000000    1.000000
Cervical neoplasm                    1.000000  0.500000  0.666667    2.000000
DNA修复异常                              1.000000  1.000000  1.000000    1.000000
Hydatidiform mole                    1.000000  1.000000  1.000000    1.000000
Labial melanotic macule              0.500000  1.000000  0.666667    1.000000
...                                       ...       ...       ...         ...
黄体生成素水平升高                            1.000000  1.000000  1.000000    2.000000
鼻咽部赘生瘤                               1.000000  1.000000  1.000000    2.000000
accuracy                             0.742424  0.742424  0.742424    0.742424
macro avg                            0.577877  0.567950  0.557653  858.000000
we

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: Undef

In [51]:
### cross validation

from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

#clf = SVC(kernel="linear", C = 0.055)
clf = MLPClassifier(alpha=0.02, max_iter=1000)

scores_a = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')
scores_f1 = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_weighted')

print("Accuracy: %0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() * 2))
print("F1: %0.2f (+/- %0.2f)" % (scores_f1.mean(), scores_f1.std() * 2))

C:\Users\Jingz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
C:\Users\Jingz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Accuracy: 0.74 (+/- 0.04)
F1: 0.71 (+/- 0.03)


In [9]:
from sklearn.linear_model import PassiveAggressiveClassifier


# Passive-Aggressive I
pac1 = Pipeline([
                ('clf', PassiveAggressiveClassifier(loss='hinge', C=0.005, tol=1e-4)),
               ])
pac1.fit(X_train, y_train)

y_pred = pac1.predict(X_test)
my_tags = pd.unique(y_pred).tolist()


# result metrics
print('accuracy %s' % metrics.accuracy_score(y_pred, y_test))   
print('recall %s'   % metrics.recall_score(y_pred, y_test,  average='weighted'))      #, labels=np.unique(y_pred)   # tp / (tp + fn)
print('precision %s' % metrics.precision_score(y_pred, y_test,  average='weighted'))   # tp / (tp + fp)
print('f1_sccore %s' % metrics.f1_score(y_pred, y_test,  average='weighted'))          # 2*(precision * recall) / (precision + recall)


# save metric results into excel
print('accuracy %s' %  metrics.accuracy_score(y_pred, y_test))
data = pd.DataFrame( metrics.classification_report(y_test, y_pred, output_dict=True)).T
print(data)
data.to_excel('training_rapport_pac1.xlsx', header=True, index=True, encoding = 'ANSI')


# save model
with open('M_pac1_classifier_cn.pickle', 'wb') as handle:
    pickle.dump(pac1, handle, protocol=pickle.HIGHEST_PROTOCOL)  

accuracy 0.7284382284382285
recall 0.7284382284382285
precision 0.7917765384865023
f1_sccore 0.7421631435631992
accuracy 0.7284382284382285
                                    precision    recall  f1-score     support
Abnormal zona pellucida morphology   1.000000  1.000000  1.000000    1.000000
Cervical neoplasm                    0.500000  0.500000  0.500000    2.000000
DNA修复异常                              1.000000  1.000000  1.000000    1.000000
Hydatidiform mole                    0.500000  1.000000  0.666667    1.000000
Labial melanotic macule              1.000000  1.000000  1.000000    1.000000
...                                       ...       ...       ...         ...
黄体生成素水平升高                            1.000000  1.000000  1.000000    2.000000
鼻咽部赘生瘤                               1.000000  1.000000  1.000000    2.000000
accuracy                             0.728438  0.728438  0.728438    0.728438
macro avg                            0.560512  0.555662  0.543493  858.000000
we

C:\Users\Jingz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jingz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jingz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jingz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: Undef

In [10]:
### (soft-margin) Linear Support Vector Machine
sgd = Pipeline([
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=100, tol=None)),
               ])
sgd.fit(X_train, y_train)

y_pred = sgd.predict(X_test)
my_tags = pd.unique(y_pred).tolist()


# result metrics
print('accuracy %s' % metrics.accuracy_score(y_pred, y_test))   
print('recall %s'   % metrics.recall_score(y_pred, y_test,  average='weighted'))      #, labels=np.unique(y_pred)   # tp / (tp + fn)
print('precision %s' % metrics.precision_score(y_pred, y_test,  average='weighted'))   # tp / (tp + fp)
print('f1_sccore %s' % metrics.f1_score(y_pred, y_test,  average='weighted'))          # 2*(precision * recall) / (precision + recall)


# save metric results into excel
data = pd.DataFrame(metrics.classification_report(y_test, y_pred, output_dict=True)).T
print(data)
data.to_excel('training_rapport_sgd.xlsx', header=True, index=True, encoding = 'ANSI')

### save model
with open('M_sgd_classifier_cn.pickle', 'wb') as handle:
    pickle.dump(sgd, handle, protocol=pickle.HIGHEST_PROTOCOL)

accuracy 0.6526806526806527
recall 0.6526806526806527
precision 0.7987976191293704
f1_sccore 0.617520895519325
                                    precision    recall  f1-score     support
Abnormal zona pellucida morphology   1.000000  1.000000  1.000000    1.000000
Cervical neoplasm                    0.000000  0.000000  0.000000    2.000000
DNA修复异常                              0.000000  0.000000  0.000000    1.000000
Hydatidiform mole                    1.000000  1.000000  1.000000    1.000000
Labial melanotic macule              1.000000  1.000000  1.000000    1.000000
...                                       ...       ...       ...         ...
黄体生成素水平升高                            1.000000  1.000000  1.000000    2.000000
鼻咽部赘生瘤                               1.000000  1.000000  1.000000    2.000000
accuracy                             0.652681  0.652681  0.652681    0.652681
macro avg                            0.535613  0.505094  0.505798  858.000000
weighted avg                   

C:\Users\Jingz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jingz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jingz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jingz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: Undef

### Load pretrained model and predict

In [27]:
with open('tokenizer_keras_cn_skl.pickle', 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)
with open('M_svm_Classifier_cn.pickle', 'rb') as handle:
    skl= pickle.load(handle)
    
y_pred = skl.predict(X_test)
i=random.randint(1, len(X_test))
print(i, 'Predicted_class:', y_pred[i], '; Real_class:', y_test.iloc[i])

#y_pred = logreg.predict(x_test)
#print(i, 'Predicted_class:', y_pred[i], '; Real_class:', y_test.iloc[i])

112 Predicted_class: 染色体断裂 ; Real_class: 染色体断裂
